In [1]:
#No51(特徴量抽出)
#特徴量 対象データの特徴を定量的な数値として表したもの
#TfidfVectorizer 単語の出現頻度(TF)と単語が文書にてどのぐらい出てこないか(IDF)で判断する。
import pandas as pd
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
drive.mount('/content/drive')
!wget https://archive.ics.uci.edu/static/public/359/news+aggregator.zip
!unzip news+aggregator.zip
df = pd.read_csv("./newsCorpora.csv",sep="\t",header=None,names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])
#「PUBLISHER」の行から、”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
#isinはあくまでbool値を返すことに注意(Trueが抽出される)
df = df[df["PUBLISHER"].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]
#「TITLE」と「CATEGORY」の列を抽出する．
df = df[["TITLE", "CATEGORY"]]
#学習、検証、評価データに分割する(分割したいもの、割合、shuffleはTrueがデフォルト)
#まず0.9:0.1で分ける
train, test = train_test_split(df, test_size=0.2)
#次に0.2を半分にする(検証、評価データを0.1にする)
test, valid = train_test_split(test, test_size=0.5)
#学習、検証、評価データをファイルに保存する
train.to_csv("train.txt", sep="\t", index=False, header=None)
valid.to_csv("valid.txt", sep="\t", index=False, header=None)
test.to_csv("test.txt", sep="\t", index=False, header=None)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--2024-06-05 14:05:04--  https://archive.ics.uci.edu/static/public/359/news+aggregator.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘news+aggregator.zip’

news+aggregator.zip     [          <=>       ]  27.87M  9.47MB/s    in 2.9s    

2024-06-05 14:05:08 (9.47 MB/s) - ‘news+aggregator.zip’ saved [29224203]

Archive:  news+aggregator.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [2]:
def preprosessing(text):
    #string.punctuation 「'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'」のこと
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    #maketransで作成したtableを文字列を変換する
    text = text.translate(table)
    #小文字にする
    text = text.lower()
    #正規表現を利用するために、compileを用いてpatternを作成する
    pattern = re.compile('[0-9]+')
    #正規表現にマッチした部分に0を代入
    text = re.sub(pattern, '0', text)

    return text

#データの連結、前処理
df = pd.concat([train, valid, test], axis = 0)
#もとのindexを削除
df.reset_index(drop=True, inplace=True)
df['TITLE'] = df['TITLE'].map(lambda x: preprosessing(x)) #map関数を使ってSeriesの各要素に前処理の関数を適用
#データの連結、前処理
df = pd.concat([train, valid, test], axis = 0)
df.reset_index(drop=True, inplace=True)
df['TITLE'] = df['TITLE'].map(lambda x: preprosessing(x)) #map関数を使ってSeriesの各要素に前処理の関数を適用

#単語のベクトル化
vec_tfidf = TfidfVectorizer() #TfidfVectorizerのインスタンス生成
data = vec_tfidf.fit_transform(df['TITLE'])
data = pd.DataFrame(data.toarray(), columns = vec_tfidf.get_feature_names_out())

#整数除算をして、dataを分割する。
split_point1 = int(len(data)//3)
split_point2 = int(split_point1 * 2)

#学習、検証、評価データ
x_train = data[:split_point1]
x_valid = data[split_point1:split_point2]
x_test = data[split_point2:]

#学習、検証、評価等別
y_data = df['CATEGORY']
y_train = y_data[:split_point1]
y_valid = y_data[split_point1:split_point2]
y_test = y_data[split_point2:]

#CSVファイルへの書き出し
x_train.to_csv('train.feature.txt', sep = '\t', index = False)
x_valid.to_csv('valid.feature.txt', sep = '\t', index = False)
x_test.to_csv('test.feature.txt', sep = '\t', index = False)
